In [7]:
import pandas as pd
import requests
from pandas import json_normalize
from io import BytesIO
import time
from datetime import datetime, timedelta

entrade_headers = {
  'authority': 'services.entrade.com.vn',
  'accept': 'application/json, text/plain, */*',
  'accept-language': 'en-US,en;q=0.9',
  'dnt': '1',
  'origin': 'https://banggia.dnse.com.vn',
  'referer': 'https://banggia.dnse.com.vn/',
  'sec-ch-ua': '"Edge";v="114", "Chromium";v="114", "Not=A?Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'cross-site',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1788.0'
}

def stock_historical_data (symbol, start_date='2023-06-01', end_date='2023-06-17', resolution='1D', type='stock', headers=entrade_headers): # DNSE source (will be published on vnstock)
    """
    Get historical price data from entrade.com.vn
    Parameters:
        symbol (str): ticker of a stock or index. Available indices are: VNINDEX, VN30, HNX, HNX30, UPCOM, VNXALLSHARE, VN30F1M, VN30F2M, VN30F1Q, VN30F2Q
        from_date (str): start date of the historical price data
        to_date (str): end date of the historical price data
        resolution (str): resolution of the historical price data. Default is '1D' (daily), other options are '1' (1 minute), 15 (15 minutes), 30 (30 minutes), '1H' (hourly)
        type (str): stock or index. Default is 'stock'
        headers (dict): headers of the request
    Returns:
        :obj:`pandas.DataFrame`:
        | time | open | high | low | close | volume |
        | ----------- | ---- | ---- | --- | ----- | ------ |
        | YYYY-mm-dd  | xxxx | xxxx | xxx | xxxxx | xxxxxx |
    """
    # convert from_date, to_date to timestamp
    from_timestamp = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
    to_timestamp = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())
    url = f"https://services.entrade.com.vn/chart-api/v2/ohlcs/{type}?from={from_timestamp}&to={to_timestamp}&symbol={symbol}&resolution={resolution}"
    response = requests.request("GET", url, headers=headers).json()
    df = pd.DataFrame(response)
    df['t'] = pd.to_datetime(df['t'], unit='s') # convert timestamp to datetime
    df = df.rename(columns={'t': 'time', 'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'}).drop(columns=['nextTime'])
    df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Ho_Chi_Minh')
    return df

df =  stock_historical_data(symbol='GMD', 
                            start_date="2023-01-01", 
                            end_date='2023-07-14', resolution='1D', type='stock')
print(df)

                         time   open   high    low  close   volume
0   2023-01-03 09:00:00+07:00  46.20  46.20  45.40  46.00   537800
1   2023-01-04 09:00:00+07:00  46.00  48.00  46.00  46.00   487100
2   2023-01-05 09:00:00+07:00  46.95  46.95  46.00  46.05   551800
3   2023-01-06 09:00:00+07:00  47.20  47.70  46.55  46.60   362500
4   2023-01-09 09:00:00+07:00  46.80  47.40  46.80  47.30   268900
..                        ...    ...    ...    ...    ...      ...
125 2023-07-07 09:00:00+07:00  53.80  56.00  53.20  55.80  2437800
126 2023-07-10 09:00:00+07:00  56.00  56.00  55.20  55.70  1137100
127 2023-07-11 09:00:00+07:00  56.00  57.90  55.90  56.70  2642800
128 2023-07-12 09:00:00+07:00  57.50  57.50  56.70  57.10   953000
129 2023-07-13 09:00:00+07:00  57.20  57.50  56.60  57.20  1025600

[130 rows x 6 columns]
